In [2]:
!pip install rdkit
!pip install mordred

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 21.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 KB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.7 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176721 sha256=678199f0bc999c90329ca91fa98f406e0077c4708492c3912a2754c4ed6177a9
  Stored in directory: /root/.cache/pip/wheels/05/95/d1/9e913738f0e8362b3676917b953a60afd76d2b0b99ff8a71ec
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0


In [3]:
import numpy as np
import pandas as pd
import rdkit
from rdkit import Chem, rdBase, DataStructs
from rdkit.Chem import rdmolfiles, rdmolops, AllChem, Draw, rdDepictor, Descriptors
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.Fingerprints import FingerprintMols

from IPython.display import SVG
from IPython.display import HTML
from IPython.display import display

from collections import defaultdict
from tqdm import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('./drive/MyDrive/AI/Material Informatics (MI)/Ring Repeating Units/Table of Supporting Information(SMILES)- Cal band gap_(RU1, RU2, RRU).csv')

In [6]:
df.head()

,NO.,Repeating units_RU1,Repeating units_RU2,SMILES_RU1,SMILES_RU2,SMILES_RRU,DFT band gap/eV
0,1,CH2-CO-NH-CS,CO-NH-CS-CH2,*-CC(=O)NC(-*)=S,*-CC(=S)NC(-*)=O,O=C1CC(=S)N1,2.691
1,2,CH2-CS-C6H4-O,CS-C6H4-O-CH2,*-CC(=S)C1=CC=C(O-*)C=C1,*-COC1=CC=C(C=C1)C(-*)=S,S=C1COC2=CC=C1C=C2,2.041
2,3,C6H4-C6H4-C4H2S-CS,C6H4-C4H2S-CS-C6H4,*-C(=S)C1=CC=C(S1)C1=CC=C(C=C1)C1=CC=C(-*)C=C1,*-C1=CC=C(C=C1)C(=S)C1=CC=C(S1)C1=CC=C(-*)C=C1,S=C1C2=CC=C(S2)C2=CC=C(C=C2)C2=CC=C1C=C2,1.807
3,4,CO-O-C4H2S-CS,O-C4H2S-CS-CO,*-C(=O)OC1=CC=C(S1)C(-*)=S,*-OC1=CC=C(S1)C(=S)C(-*)=O,O=C1OC2=CC=C(S2)C1=S,1.918
4,5,C6H4-CS-C4H2S-O,CS-C4H2S-O-C6H4,*-OC1=CC=C(S1)C(=S)C1=CC=C(-*)C=C1,*-C(=S)C1=CC=C(OC2=CC=C(-*)C=C2)S1,S=C1C2=CC=C(OC3=CC=C1C=C3)S2,1.930


In [7]:
df["mol_ru1"] = df["SMILES_RU1"].apply(lambda x:Chem.MolFromSmiles(x))
df["mol_ru2"] = df["SMILES_RU2"].apply(lambda x:Chem.MolFromSmiles(x))
df["mol_rru"] = df["SMILES_RRU"].apply(lambda x:Chem.MolFromSmiles(x))

In [8]:
df.head()

,NO.,Repeating units_RU1,Repeating units_RU2,SMILES_RU1,SMILES_RU2,SMILES_RRU,DFT band gap/eV,mol_ru1,mol_ru2,mol_rru
0,1,CH2-CO-NH-CS,CO-NH-CS-CH2,*-CC(=O)NC(-*)=S,*-CC(=S)NC(-*)=O,O=C1CC(=S)N1,2.691,<rdkit.Chem.rdchem.Mol object at 0x7f4d884df350>,<rdkit.Chem.rdchem.Mol object at 0x7f4d884ef0b0>,<rdkit.Chem.rdchem.Mol object at 0x7f4d884faeb0>
1,2,CH2-CS-C6H4-O,CS-C6H4-O-CH2,*-CC(=S)C1=CC=C(O-*)C=C1,*-COC1=CC=C(C=C1)C(-*)=S,S=C1COC2=CC=C1C=C2,2.041,<rdkit.Chem.rdchem.Mol object at 0x7f4d884df270>,<rdkit.Chem.rdchem.Mol object at 0x7f4d884ef190>,<rdkit.Chem.rdchem.Mol object at 0x7f4d884faf90>
2,3,C6H4-C6H4-C4H2S-CS,C6H4-C4H2S-CS-C6H4,*-C(=S)C1=CC=C(S1)C1=CC=C(C=C1)C1=CC=C(-*)C=C1,*-C1=CC=C(C=C1)C(=S)C1=CC=C(S1)C1=CC=C(-*)C=C1,S=C1C2=CC=C(S2)C2=CC=C(C=C2)C2=CC=C1C=C2,1.807,<rdkit.Chem.rdchem.Mol object at 0x7f4d884df200>,<rdkit.Chem.rdchem.Mol object at 0x7f4d884ef120>,<rdkit.Chem.rdchem.Mol object at 0x7f4d884faf20>
3,4,CO-O-C4H2S-CS,O-C4H2S-CS-CO,*-C(=O)OC1=CC=C(S1)C(-*)=S,*-OC1=CC=C(S1)C(=S)C(-*)=O,O=C1OC2=CC=C(S2)C1=S,1.918,<rdkit.Chem.rdchem.Mol object at 0x7f4d884df2e0>,<rdkit.Chem.rdchem.Mol object at 0x7f4d884ef200>,<rdkit.Chem.rdchem.Mol object at 0x7f4d884f3040>
4,5,C6H4-CS-C4H2S-O,CS-C4H2S-O-C6H4,*-OC1=CC=C(S1)C(=S)C1=CC=C(-*)C=C1,*-C(=S)C1=CC=C(OC2=CC=C(-*)C=C2)S1,S=C1C2=CC=C(OC3=CC=C1C=C3)S2,1.930,<rdkit.Chem.rdchem.Mol object at 0x7f4d884df3c0>,<rdkit.Chem.rdchem.Mol object at 0x7f4d884ef270>,<rdkit.Chem.rdchem.Mol object at 0x7f4d884f30b0>


In [9]:
#RDKit fingerprints
def rdkit_fingerprint(mols):
  fp = [AllChem.RDKFingerprint(mol) for mol in mols]
  df_fp = pd.DataFrame(np.array(fp, int))
  return df_fp

In [11]:
ru1 = rdkit_fingerprint(df["mol_ru1"])
ru1 = ru1.dropna(1)

ru2 = rdkit_fingerprint(df["mol_ru2"])
ru2 = ru2.dropna(1)

rru = rdkit_fingerprint(df["mol_rru"])
rru = rru.dropna(1)

<ipython-input-11-235708a0c4f5>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  ru1 = ru1.dropna(1)
<ipython-input-11-235708a0c4f5>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  ru2 = ru2.dropna(1)
<ipython-input-11-235708a0c4f5>:8: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  rru = rru.dropna(1)


In [12]:
ru1

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
280,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
281,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
282,0,1,1,0,0,0,0,0,0,0,...,1,0,0,1,1,1,0,0,0,0


In [13]:
# A list for the evaluation result
rmse_sum = {}
r2_sum = {}

RU1

In [14]:
from sklearn.model_selection import train_test_split

X = ru1.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [15]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [16]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_ru1: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_ru1: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_ru1: 0.6467181307897243
RMSE(Train): 0.7234570660334166

R_square(Test)_ru1: 0.6603649581694263
RMSE(Test): 0.6125628437882477


In [17]:
rmse_sum['RU1'] = [RMSE_train, RMSE_test] 
r2_sum['RU1'] = [R_square_train, R_square_test] 

RU2

In [18]:
from sklearn.model_selection import train_test_split

X = ru2.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [19]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [20]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_ru2: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_ru2: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_ru2: 0.6357630206558342
RMSE(Train): 0.7345884672644585

R_square(Test)_ru2: 0.6565274195309521
RMSE(Test): 0.6160137989391646


In [21]:
rmse_sum['RU2'] = [RMSE_train, RMSE_test] 
r2_sum['RU2'] = [R_square_train, R_square_test] 

RRU

In [22]:
from sklearn.model_selection import train_test_split

X = rru.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [23]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [24]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_rru: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_rru: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_rru: 0.504557804733397
RMSE(Train): 0.8567390919038352

R_square(Test)_rru: 0.5622955846341855
RMSE(Test): 0.6954002908348257


In [25]:
rmse_sum['RRU'] = [RMSE_train, RMSE_test] 
r2_sum['RRU'] = [R_square_train, R_square_test] 

Summary

In [26]:
summary = pd.DataFrame(r2_sum, index=["train", "test"])
summary.style.set_caption("Table of R Square (RDKit fingerprints)")

,RU1,RU2,RRU
train,0.646718,0.635763,0.504558
test,0.660365,0.656527,0.562296


In [27]:
summary = pd.DataFrame(rmse_sum, index=["train", "test"])
summary.style.set_caption("Table of RMSE (RDKit fingerprints)")

,RU1,RU2,RRU
train,0.723457,0.734588,0.856739
test,0.612563,0.616014,0.695400
